In [1]:
# imports

import os
# import urllib2
import pandas as pd
import urllib.request, json
# from pprint import pprint
from pandas.io.json import json_normalize
# import matplotlib.pyplot as plt
import datetime

In [2]:
# load api from covidtracking in json form

with urllib.request.urlopen("https://covidtracking.com/api/v1/states/daily.json") as url:
    states_historical = json.loads(url.read().decode())
#     pprint(states_historical)

In [3]:
# jsonify as pandas df

state_historical_df = json_normalize(states_historical[0:]).fillna(0)
state_historical_df

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200430,AK,355.0,18764.0,0.0,19.0,0.0,0.0,0.0,0.0,...,0.0,19119.0,19119.0,19119.0,02,0.0,0.0,0.0,0.0,0.0
1,20200430,AL,7019.0,80177.0,0.0,0.0,978.0,0.0,335.0,0.0,...,978.0,87196.0,87196.0,87196.0,01,24.0,33.0,6570.0,177.0,6747.0
2,20200430,AR,3255.0,45124.0,0.0,95.0,402.0,0.0,0.0,23.0,...,402.0,48379.0,48379.0,48379.0,05,2.0,13.0,2867.0,63.0,2930.0
3,20200430,AS,0.0,3.0,17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200430,AZ,7648.0,64138.0,0.0,755.0,1169.0,311.0,0.0,194.0,...,1169.0,71786.0,71786.0,71786.0,04,16.0,38.0,2527.0,446.0,2973.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148,20200126,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,53,0.0,0.0,0.0,0.0,0.0
3149,20200125,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,53,0.0,0.0,0.0,0.0,0.0
3150,20200124,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,53,0.0,0.0,0.0,0.0,0.0
3151,20200123,WA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,53,0.0,0.0,0.0,0.0,0.0


In [4]:
# load csv file with state populations and calculate population per millions

popul = os.path.join('./Resources/population.csv')
popul_df = pd.read_csv(popul)
popul_df.columns = ['state', 'population', 'pop (mil)']
# popul_df.dtypes
popul_df.head()

,state,population,pop (mil)
0,AL,"4,903,185",4.90
1,AK,"731,545",0.73
2,AZ,"7,278,717",7.28
3,AR,"3,017,804",3.02
4,CA,"39,512,223",39.51


In [5]:
# merge two df

conct_df = pd.merge(state_historical_df,
                 popul_df[['state', 'population', 'pop (mil)']], on = 'state')

# conct_df.head()

In [6]:
# sort by state (leaving dates until after daily avgs are calculated)

sorted_hist_df = conct_df.sort_values(by=['state', 'date'], ascending=[True, True])
sorted_hist_df = sorted_hist_df.reset_index(drop=True)
# sorted_hist_df

In [7]:
# sort new df by columns, rename columns, convert date and reindex

# sorted_df = state_historical_df.loc[state_historical_df['state'] == state]
sorted_hist_df = sorted_hist_df[['date', 'state', 'population', 'pop (mil)', 'positive', 'positiveIncrease','negative', 'negativeIncrease', 
                     'totalTestResults', 'totalTestResultsIncrease', 'hospitalized', 'hospitalizedIncrease',
                     'death', 'deathIncrease']].fillna(0)

# sorted_hist_df = sorted_hist_df.iloc[::-1]
sorted_hist_df['date'] = pd.to_datetime(sorted_hist_df['date'].astype(str), format='%Y%m%d')
sorted_hist_df = sorted_hist_df.reset_index(drop=True)

sorted_hist_df.rename(columns = {'date':'Date', 'state':'State', 'population':'Population', 'pop (mil)':'Pop (mil)', 'positive':'Total Positive Cases', 'positiveIncrease':'Positives Increase',
                           'negative':'Negative Cases', 'negativeIncrease':'Negatives Increase', 'totalTestResults':'Total Tests',
                           'totalTestResultsIncrease':'Total Tests Increase', 'hospitalized':'Total Hospitalized',
                           'hospitalizedIncrease':'Hospitalized Increase', 'death':'Total Deaths', 'deathIncrease':'Deaths Increase'}, inplace = True)
# state_df.drop(['date'])

# sorted_hist_df

In [8]:
# Calculate and add 3 day positives increase avg using for loop

sorted_hist_df.insert(loc=6, column='3 Day Pos Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Pos Inc Avg'] = round((sorted_hist_df.loc[i, 'Positives Increase'] + 
    sorted_hist_df.loc[i-1, 'Positives Increase'] + 
    sorted_hist_df.loc[i-2, 'Positives Increase'])/3,2)
    
# zero out first 2 days of 3 day increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):
    
    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Pos Inc Avg'] = 0
    
sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,12.0,4.0,12.0,4.0,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,14.0,2.0,14.0,2.0,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,23.0,9.0,23.0,9.0,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,23.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,2020-04-26,WY,"578,759",0.58,362.0,13.0,12,7797.0,0.0,8159.0,13.0,54.0,0.0,7.0,0.0
2919,2020-04-27,WY,"578,759",0.58,370.0,8.0,7,7797.0,0.0,8167.0,8.0,54.0,0.0,7.0,0.0
2920,2020-04-28,WY,"578,759",0.58,389.0,19.0,13.33,8226.0,429.0,8615.0,448.0,56.0,2.0,7.0,0.0
2921,2020-04-29,WY,"578,759",0.58,536.0,147.0,58,8681.0,455.0,9217.0,602.0,56.0,0.0,7.0,0.0


In [9]:
# Calculate and add 7 day positives increase avg using for loop


sorted_hist_df.insert(loc=7, column='7 Day Pos Inc Avg', value ='')

for i in range(7, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '7 Day Pos Inc Avg'] = round((sorted_hist_df.loc[i, 'Positives Increase'] + 
    sorted_hist_df.loc[i-1, 'Positives Increase'] + 
    sorted_hist_df.loc[i-2, 'Positives Increase'] +
    sorted_hist_df.loc[i-3, 'Positives Increase'] + 
    sorted_hist_df.loc[i-4, 'Positives Increase'] +
    sorted_hist_df.loc[i-5, 'Positives Increase'] + 
    sorted_hist_df.loc[i-6, 'Positives Increase'])/7,2)
    
# zero out first 7 days of 7 day increase avg using for loop - new state was calculating off of previous states last days
for i in range(7, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+2, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+3, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+4, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+5, '7 Day Pos Inc Avg'] = 0
        sorted_hist_df.loc[i+6, '7 Day Pos Inc Avg'] = 0

sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12.0,4.0,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14.0,2.0,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23.0,9.0,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,2020-04-26,WY,"578,759",0.58,362.0,13.0,12,7.57,7797.0,0.0,8159.0,13.0,54.0,0.0,7.0,0.0
2919,2020-04-27,WY,"578,759",0.58,370.0,8.0,7,8.14,7797.0,0.0,8167.0,8.0,54.0,0.0,7.0,0.0
2920,2020-04-28,WY,"578,759",0.58,389.0,19.0,13.33,9.86,8226.0,429.0,8615.0,448.0,56.0,2.0,7.0,0.0
2921,2020-04-29,WY,"578,759",0.58,536.0,147.0,58,30.57,8681.0,455.0,9217.0,602.0,56.0,0.0,7.0,0.0


In [10]:
sorted_hist_df[265:280
            ]

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
265,2020-04-13,CA,"39,512,223",39.51,22348.0,554.0,958.67,1144.57,168534.0,0.0,190882.0,554.0,0.0,0.0,687.0,36.0
266,2020-04-14,CA,"39,512,223",39.51,23338.0,990.0,1288.67,1067.57,178870.0,10336.0,202208.0,11326.0,0.0,0.0,758.0,71.0
267,2020-04-15,CA,"39,512,223",39.51,24424.0,1086.0,876.67,1066.71,192062.0,13192.0,216486.0,14278.0,0.0,0.0,821.0,63.0
268,2020-04-16,CA,"39,512,223",39.51,26182.0,1758.0,1278,1124.71,220218.0,28156.0,246400.0,29914.0,0.0,0.0,890.0,69.0
269,2020-04-17,CA,"39,512,223",39.51,27528.0,1346.0,1396.67,1150.86,224086.0,3868.0,251614.0,5214.0,0.0,0.0,985.0,95.0
270,2020-04-18,CA,"39,512,223",39.51,28963.0,1435.0,1513,1355.86,230703.0,6617.0,259666.0,8052.0,0.0,0.0,1072.0,87.0
271,2020-04-19,CA,"39,512,223",39.51,30333.0,1370.0,1383.67,1219.86,250567.0,19864.0,280900.0,21234.0,0.0,0.0,1166.0,94.0
272,2020-04-20,CA,"39,512,223",39.51,30978.0,645.0,1150,1232.86,259522.0,8955.0,290500.0,9600.0,0.0,0.0,1208.0,42.0
273,2020-04-21,CA,"39,512,223",39.51,33261.0,2283.0,1432.67,1417.57,266839.0,7317.0,300100.0,9600.0,0.0,0.0,1268.0,60.0
274,2020-04-22,CA,"39,512,223",39.51,35396.0,2135.0,1687.67,1567.43,429931.0,163092.0,465327.0,165227.0,0.0,0.0,1354.0,86.0


In [11]:
sorted_hist_df.columns

Index(['Date', 'State', 'Population', 'Pop (mil)', 'Total Positive Cases',
       'Positives Increase', '3 Day Pos Inc Avg', '7 Day Pos Inc Avg',
       'Negative Cases', 'Negatives Increase', 'Total Tests',
       'Total Tests Increase', 'Total Hospitalized', 'Hospitalized Increase',
       'Total Deaths', 'Deaths Increase'],
      dtype='object')

In [12]:
# Calculate and add 3 day total testing avg using for loop

sorted_hist_df.insert(loc=12, column='3 Day Testing Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Testing Inc Avg'] = round((sorted_hist_df.loc[i, 'Total Tests Increase'] + 
    sorted_hist_df.loc[i-1, 'Total Tests Increase'] + 
    sorted_hist_df.loc[i-2, 'Total Tests Increase'])/3,2)
    
# zero out first 2 days of 3 day testing increase avg using for loop - new state was calculating off of previous states last days    
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Testing Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Testing Inc Avg'] = 0
        
sorted_hist_df.fillna(0)
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8.0,0.0,,0.0,0.0,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12.0,4.0,,0.0,0.0,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14.0,2.0,2,0.0,0.0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23.0,9.0,5,0.0,0.0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23.0,0.0,3.67,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,2020-04-26,WY,"578,759",0.58,362.0,13.0,12,7.57,7797.0,0.0,8159.0,13.0,197.33,54.0,0.0,7.0,0.0
2919,2020-04-27,WY,"578,759",0.58,370.0,8.0,7,8.14,7797.0,0.0,8167.0,8.0,40.67,54.0,0.0,7.0,0.0
2920,2020-04-28,WY,"578,759",0.58,389.0,19.0,13.33,9.86,8226.0,429.0,8615.0,448.0,156.33,56.0,2.0,7.0,0.0
2921,2020-04-29,WY,"578,759",0.58,536.0,147.0,58,30.57,8681.0,455.0,9217.0,602.0,352.67,56.0,0.0,7.0,0.0


In [13]:
sorted_hist_df.columns

Index(['Date', 'State', 'Population', 'Pop (mil)', 'Total Positive Cases',
       'Positives Increase', '3 Day Pos Inc Avg', '7 Day Pos Inc Avg',
       'Negative Cases', 'Negatives Increase', 'Total Tests',
       'Total Tests Increase', '3 Day Testing Inc Avg', 'Total Hospitalized',
       'Hospitalized Increase', 'Total Deaths', 'Deaths Increase'],
      dtype='object')

In [14]:
# Calculate and add 3 day hospitalized inc avg using for loop


sorted_hist_df.insert(loc=15, column='3 Day Hospitalized Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Hospitalized Inc Avg'] = round((sorted_hist_df.loc[i, 'Hospitalized Increase'] + 
    sorted_hist_df.loc[i-1, 'Hospitalized Increase'] + 
    sorted_hist_df.loc[i-2, 'Hospitalized Increase'])/3,2)
    
# zero out first 2 days of 3 day hospitalized increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Hospitalized Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Hospitalized Inc Avg'] = 0

sorted_hist_df
    
# sorted_hist_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,3 Day Hospitalized Inc Avg,Total Deaths,Deaths Increase
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8.0,0.0,,0.0,0.0,,0.0,0.0
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12.0,4.0,,0.0,0.0,,0.0,0.0
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14.0,2.0,2,0.0,0.0,0,0.0,0.0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23.0,9.0,5,0.0,0.0,0,0.0,0.0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23.0,0.0,3.67,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,2020-04-26,WY,"578,759",0.58,362.0,13.0,12,7.57,7797.0,0.0,8159.0,13.0,197.33,54.0,0.0,0.67,7.0,0.0
2919,2020-04-27,WY,"578,759",0.58,370.0,8.0,7,8.14,7797.0,0.0,8167.0,8.0,40.67,54.0,0.0,0,7.0,0.0
2920,2020-04-28,WY,"578,759",0.58,389.0,19.0,13.33,9.86,8226.0,429.0,8615.0,448.0,156.33,56.0,2.0,0.67,7.0,0.0
2921,2020-04-29,WY,"578,759",0.58,536.0,147.0,58,30.57,8681.0,455.0,9217.0,602.0,352.67,56.0,0.0,0.67,7.0,0.0


In [15]:
# Calculate and add 3 day deaths inc avg using for loop


sorted_hist_df.insert(loc=18, column='3 Day Deaths Inc Avg', value ='')

for i in range(2, len(sorted_hist_df)):
    sorted_hist_df.loc[i, '3 Day Deaths Inc Avg'] = round((sorted_hist_df.loc[i, 'Deaths Increase'] + 
    sorted_hist_df.loc[i-1, 'Deaths Increase'] + 
    sorted_hist_df.loc[i-2, 'Deaths Increase'])/3,2)

# zero out first 2 days of 3 day deaths increase avg using for loop - new state was calculating off of previous states last days
for i in range(2, len(sorted_hist_df)):

    if sorted_hist_df.loc[i, 'State'] != sorted_hist_df.loc[i-1, 'State']:
        sorted_hist_df.loc[i, '3 Day Deaths Inc Avg'] = 0
        sorted_hist_df.loc[i+1, '3 Day Deaths Inc Avg'] = 0
        
sorted_hist_df.fillna(0)
    


,Date,State,Population,Pop (mil),Total Positive Cases,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Negative Cases,Negatives Increase,Total Tests,Total Tests Increase,3 Day Testing Inc Avg,Total Hospitalized,Hospitalized Increase,3 Day Hospitalized Inc Avg,Total Deaths,Deaths Increase,3 Day Deaths Inc Avg
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,,,8.0,0.0,8.0,0.0,,0.0,0.0,,0.0,0.0,
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,,,12.0,4.0,12.0,4.0,,0.0,0.0,,0.0,0.0,
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0,,14.0,2.0,14.0,2.0,2,0.0,0.0,0,0.0,0.0,0
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0,,23.0,9.0,23.0,9.0,5,0.0,0.0,0,0.0,0.0,0
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0,,23.0,0.0,23.0,0.0,3.67,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,2020-04-26,WY,"578,759",0.58,362.0,13.0,12,7.57,7797.0,0.0,8159.0,13.0,197.33,54.0,0.0,0.67,7.0,0.0,0
2919,2020-04-27,WY,"578,759",0.58,370.0,8.0,7,8.14,7797.0,0.0,8167.0,8.0,40.67,54.0,0.0,0,7.0,0.0,0
2920,2020-04-28,WY,"578,759",0.58,389.0,19.0,13.33,9.86,8226.0,429.0,8615.0,448.0,156.33,56.0,2.0,0.67,7.0,0.0,0
2921,2020-04-29,WY,"578,759",0.58,536.0,147.0,58,30.57,8681.0,455.0,9217.0,602.0,352.67,56.0,0.0,0.67,7.0,0.0,0


In [16]:
# calculations

sorted_hist_df.insert(loc=8, column='Positive Cases per Mil', value ='')
sorted_hist_df.insert(loc=14, column='Tests per Mil', value ='')
sorted_hist_df.insert(loc=18, column='Hospitalized per Mil', value ='')
sorted_hist_df.insert(loc=22, column='Deaths per Mil', value ='')
sorted_hist_df.insert(loc=12, column='Testing %', value ='')
sorted_hist_df.insert(loc=5, column='Positive Cases %', value ='')
sorted_hist_df.insert(loc=17, column='Tests per Mil/Positive Cases', value ='')
sorted_hist_df.insert(loc=18, column='Tests per Mil/Deaths', value ='')
sorted_hist_df.insert(loc=27, column='Deaths/Total Tests', value ='')
sorted_hist_df.insert(loc=28, column='Deaths/Positive Cases', value ='')
sorted_hist_df.insert(loc=20, column='Hospitalized %', value ='')
sorted_hist_df.insert(loc=25, column='Death %', value ='')


sorted_hist_df['Positive Cases per Mil'] = round(sorted_hist_df['Total Positive Cases']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Tests per Mil'] = round(sorted_hist_df['Total Tests']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Hospitalized per Mil'] = round(sorted_hist_df['Total Hospitalized']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Deaths per Mil'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)'],2)
sorted_hist_df['Testing %'] = round(sorted_hist_df['Total Tests']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Positive Cases %'] = round(sorted_hist_df['Total Positive Cases']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Tests per Mil/Positive Cases'] = round(sorted_hist_df['Tests per Mil']/sorted_hist_df['Total Positive Cases'],2)
sorted_hist_df['Tests per Mil/Deaths'] = round(sorted_hist_df['Tests per Mil']/sorted_hist_df['Total Deaths'],2)
sorted_hist_df['Deaths/Total Tests'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Total Tests']*100,2)
sorted_hist_df['Deaths/Positive Cases'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Total Positive Cases']*100,2)
sorted_hist_df['Hospitalized %'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)']/10000,3)
sorted_hist_df['Death %'] = round(sorted_hist_df['Total Deaths']/sorted_hist_df['Pop (mil)']/10000,3)

sorted_hist_df.head()

,Date,State,Population,Pop (mil),Total Positive Cases,Positive Cases %,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Positive Cases per Mil,...,Hospitalized Increase,3 Day Hospitalized Inc Avg,Hospitalized per Mil,Total Deaths,Death %,Deaths Increase,3 Day Deaths Inc Avg,Deaths per Mil,Deaths/Total Tests,Deaths/Positive Cases
0,2020-03-06,AK,"731,545",0.73,0.0,0.0,0.0,,,0.0,...,0.0,,0.0,0.0,0.0,0.0,,0.0,0.0,NaN
1,2020-03-07,AK,"731,545",0.73,0.0,0.0,0.0,,,0.0,...,0.0,,0.0,0.0,0.0,0.0,,0.0,0.0,NaN
2,2020-03-08,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN
3,2020-03-09,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN
4,2020-03-10,AK,"731,545",0.73,0.0,0.0,0.0,0,,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN


In [17]:
# resort df by descending date by state after all calculations

sorted_hist_df = sorted_hist_df.sort_values(by=['State', 'Date'], ascending=[True, False])
sorted_hist_df = sorted_hist_df.reset_index(drop=True)

# sorted_hist_df

In [18]:
# create datestamp for saving file daily

now = datetime.datetime.now()
# print ("Current date and time : ")
Previous_Date = datetime.datetime.today() - datetime.timedelta(days=1)
today = now.strftime("%Y-%m-%d")
yesterday = Previous_Date.strftime("%Y-%m-%d")
print(yesterday)

2020-04-29


In [24]:
# use todays date and sort by state to make daily df

# print(df.loc[df['A'] == 'foo'])
# state_update_df = sorted_hist_df.loc[sorted_hist_df['Date'] == 'today']
state_update_df = sorted_hist_df.loc[sorted_hist_df['Date'] == yesterday]

states50_COVID19_df = state_update_df.reset_index(drop=True)

states50_COVID19_df

,Date,State,Population,Pop (mil),Total Positive Cases,Positive Cases %,Positives Increase,3 Day Pos Inc Avg,7 Day Pos Inc Avg,Positive Cases per Mil,...,Hospitalized Increase,3 Day Hospitalized Inc Avg,Hospitalized per Mil,Total Deaths,Death %,Deaths Increase,3 Day Deaths Inc Avg,Deaths per Mil,Deaths/Total Tests,Deaths/Positive Cases
0,2020-04-29,AK,"731,545",0.73,355.0,0.049,4.0,4.67,2.86,486.30,...,0.0,0,0.00,9.0,0.001,0.0,0,12.33,0.05,2.54
1,2020-04-29,AL,"4,903,185",4.90,6842.0,0.140,155.0,190.67,196.71,1396.33,...,34.0,33.33,192.86,245.0,0.005,3.0,9.67,50.00,0.30,3.58
2,2020-04-29,AR,"3,017,804",3.02,3192.0,0.106,81.0,83.67,130.86,1056.95,...,389.0,129.67,128.81,59.0,0.002,7.0,3.33,19.54,0.13,1.85
3,2020-04-29,AZ,"7,278,717",7.28,7202.0,0.099,254.0,225.33,249,989.29,...,36.0,31.33,155.36,304.0,0.004,11.0,9.67,41.76,0.44,4.22
4,2020-04-29,CA,"39,512,223",39.51,46500.0,0.118,1469.0,1445.33,1586.29,1176.92,...,0.0,0,0.00,1887.0,0.005,78.0,59,47.76,0.31,4.06
5,2020-04-29,CO,"5,758,736",5.76,14735.0,0.256,856.0,589,612.57,2558.16,...,125.0,66.67,453.12,760.0,0.013,54.0,29.33,131.94,1.09,5.16
6,2020-04-29,CT,"3,565,287",3.57,26767.0,0.750,455.0,499.33,614,7497.76,...,0.0,0,0.00,2168.0,0.061,79.0,81.33,607.28,2.29,8.10
7,2020-04-29,DC,"705,749",0.71,4106.0,0.578,112.0,88.33,128.57,5783.10,...,0.0,0,0.00,205.0,0.029,15.0,9,288.73,1.07,4.99
8,2020-04-29,DE,"973,764",0.97,4655.0,0.480,80.0,207,207.86,4798.97,...,0.0,0,0.00,144.0,0.015,7.0,8,148.45,0.67,3.09
9,2020-04-29,FL,"21,477,737",21.48,33193.0,0.155,347.0,555,697.71,1545.30,...,358.0,156.33,261.82,1240.0,0.006,86.0,48.67,57.73,0.33,3.74


In [20]:
# # save to file - will import to tableau

sorted_hist_df.to_csv('./Resources/COVID19.csv')

MN_daily = sorted_hist_df.loc[sorted_hist_df['State'] == 'MN']
WI_daily = sorted_hist_df.loc[sorted_hist_df['State'] == 'WI']

MN_daily.to_csv('./Resources/MN_COVID19.csv')
WI_daily.to_csv('./Resources/WI_COVID19.csv')

states50_COVID19_df.to_csv('./Resources/50states_COVID19.csv')



In [21]:
# d1 = state_df['Deaths']
# d2 = state_df['Positive Cases']
# t = state_df['Date']

# plt.plot(t, d1, "-b", label="Deaths")
# plt.plot(t, d2, "-r", label="Positive Cases")
# plt.legend(loc="upper left")
# plt.title(state + ' Positive Covid-19 Cases and Deaths')
# # plt.set_xticklabels(t, rotation = 45)
# plt.xticks(rotation=45)

# plt.show()